## Uniform Buffers

In [2]:
import ipywebgl
import numpy as np

In [3]:
w = ipywebgl.GLViewer()
w.clear_color(.8, .8, .8 ,1)
w.clear()
w.execute_commands(execute_once=True)

In [4]:
program = w.create_program_ext(
"""#version 300 es

uniform UData {
  float u_scale;
  vec2 u_position;
};

in vec4 in_position;

void main() {
    gl_Position = vec4((in_position.xy * u_scale) + u_position, in_position.zw);
}
""",
    
"""#version 300 es
precision highp float;

out vec4 outColor;
 
void main() {
  outColor = vec4(1, 0, 0.5, 1);
}
""")
program

GLResourceWidget(uid=2)

In [5]:
vbo = w.create_buffer_ext(
    src_data=np.array(
        [ 0, 0,
          0, 0.5,
          0.7, 0,
        ], dtype=np.float32)
)

vao = w.create_vertex_array_ext(
    program,
    [
        (vbo, '2f32', 'in_position'),
    ]
)

In [6]:
ubo = w.create_uniform_buffer_ext(program, 'UData', 'DYNAMIC_DRAW')
ubo

GLResourceWidget(uid=5)

In [7]:
# bind to buffer array 1 because 0 is used by the ipywebgl
w.bind_buffer_base('UNIFORM_BUFFER', 1, ubo)

# and link the buffer to the program
w.uniform_block_binding(program, 'UData', 1)

w.execute_commands(execute_once=True)

In [8]:
# update the buffer data
w.bind_buffer('UNIFORM_BUFFER', ubo)
w.buffer_sub_data('UNIFORM_BUFFER', 'u_scale', np.array([2], dtype=np.float32), 0)
w.buffer_sub_data('UNIFORM_BUFFER', 'u_position', np.array([-0.5, 0.1], dtype=np.float32), 0)
w.bind_buffer('UNIFORM_BUFFER', None)

w.execute_commands(execute_once=True)

In [9]:
w.clear()

w.use_program(program)
w.bind_vertex_array(vao)
w.draw_arrays('TRIANGLES', 0, 3)

# render in loop if needed
w.execute_commands()
w

GLViewer(camera_pos=[0, 50, 200])